In [1]:
import os
import sys
from pathlib import Path

import cv2
import numpy as np
import pandas as pd
import shutil

sys.path.insert(0, "\\".join(os.path.abspath(os.curdir).split("\\")[:-2:]))

from src.utils.utils import *

## 1. MammoTherm

In [ ]:
folders_names = ['Cisto', 'Lesao Benigna', 'Lesao Maligna', 'Sem lesao']
path = "D:\\mathe\\Documents\\banco_dados_ic\\termografia\\MammoTherm"
metadata = {'id': [], 'image_path': [], 'pathology': [], 'image_view': []}

for folder_name in folders_names:
    # 
    folder_path = path + '\\' + folder_name + '\\'
    directory = Path(folder_path)
    images_path = list(directory.rglob("*.jpg*"))
    
    for image_path in images_path:
        # Extrair informações id, path e patologia do nome da imagem
        image_name = str(image_path).split("\\")[-1].replace('.jpg', '')
        metadata['id'].append(image_name)
        metadata['image_path'].append(str(image_path).replace('\\', '/'))
        metadata['pathology'].append(folder_name.replace('Lesao ', '').replace('Sem lesao', 'Normal'))
        
        # Extrair informações da view da imagem
        if 'LEMD' in image_name: metadata['image_view'].append('LEMD')
        elif 'LEME' in image_name: metadata['image_view'].append('LEME')
        elif 'LIMD' in image_name: metadata['image_view'].append('LIMD')
        elif 'LIME' in image_name: metadata['image_view'].append('LIME')
        elif 'MD' in image_name: metadata['image_view'].append('MD')
        elif 'ME' in image_name: metadata['image_view'].append('ME')
        elif 'T1' in image_name: metadata['image_view'].append('T1')
        elif 'T2' in image_name: metadata['image_view'].append('T2')
        
metadata = pd.DataFrame(metadata)
images_size = get_images_size(path, "jpg", True)
metadata['image_size_mb'] = images_size

metadata.to_csv('../../outputs/termografia/metadata_mammotherm.csv', index=False)

## DMRI

In [9]:
path = 'D:\\mathe\\Documents\\banco_dados_ic\\termografia\\DMRI'
directory = Path(path)
images_path = list(directory.rglob("*.txt*"))
rois_path = list(directory.rglob("*.png*"))
metadata = {'id': [], 'study_id': [], 'pathology': [], 'image_path': [], 'roi_image_path': []}

for image_path in images_path:
    # Extrai do nome da imagem o id e o estudo
    image_name = str(image_path).split("\\")[-1].replace('.txt', '')
    study_id = str(image_path).split("\\")[-3]
    
    # Salva as informações do id e study_id nas listas correspondentes
    metadata['id'].append(image_name)
    metadata['study_id'].append(study_id)
    
    # Cria uma lista para as imagens do estudo e pega o path do estudo
    img = []
    folder_path = f'{path}_processado/{study_id}/original'
    
    # Cria a pasta do estudo caso ela não exista
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    # Se a imagem não existir, cria a imagem
    if f'{image_name}.png' not in str(os.listdir(folder_path)):
        # Abre o arquivo de texto e salva a imagem
        with open(image_path, 'r') as fd:
            for line in fd:
                tmp = line.replace("\t", " ").replace(",", ".").split(" ")
                tmp = [float(i) for i in tmp]
                img.append(tmp)
            
            # Cria a imagem e salva
            img = np.array(img) 
            cv2.imwrite(folder_path + f'/{image_name}.png')
    
    # Salva o path da imagem na lista
    metadata['image_path'].append(folder_path + f'/{image_name}.png')
    
    # Verifica a patologia da imagem e salva na lista
    if 'DOENTES' in str(image_path):
        metadata['pathology'].append('Doente')
    else:
        metadata['pathology'].append('Normal')
    
    has_roi = False
    for roi in rois_path:
        if image_name in str(roi):
            roi_name = str(roi).split("\\")[-1]
            roi_folder_path = folder_path.replace("original", "roi")
            
            # Cria a pasta do estudo caso ela não exista
            if not os.path.exists(roi_folder_path):
                os.makedirs(roi_folder_path)
            
            # Se a imagem não existir, copia a imagem da pasta original
            if f'{roi_name}' not in os.listdir(roi_folder_path):
                shutil.copy(str(roi), roi_folder_path + f'/{roi_name}')
            metadata['roi_image_path'].append(roi_folder_path + f'/{roi_name}')
            has_roi = True
            break

    # Se não tiver roi, adiciona um valor nulo
    if not has_roi:
        metadata['roi_image_path'].append(np.nan)
        
metadata = pd.DataFrame(metadata)

# Adiciona o tamanho das imagens
images_size = []
for index, row in metadata.iterrows():
    images_size.append(get_images_size(row['image_path'], "png", False))
metadata['image_size_mb'] = images_size

metadata.to_csv('../../outputs/termografia/metadata_dmri.csv', index=False)